In [38]:
import pickle
import numpy as np

data_vocab_path = "../data/jigsaw/data_vocab.bin"

In [39]:
vocab=pickle.load(open(data_vocab_path,'rb'))

In [40]:
from pytorch_pretrained_bert.tokenization import BertTokenizer

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [41]:
bert_vocab_toks = list(bert_tokenizer.vocab.keys())
vocab_toks = set( [w for idx, w in vocab.get_index_to_token_vocabulary().items() ])
len(vocab_toks), len(bert_vocab_toks)

(305140, 30522)

In [42]:
import nmslib, time

M = 25
efC = 200

num_threads = 8
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}
print('Index-time parameters', index_time_params)

Index-time parameters {'M': 25, 'indexThreadQty': 8, 'efConstruction': 200, 'post': 0}


In [43]:
# Space name should correspond to the space name 
# used for brute-force search
space_name='leven'


# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space=space_name, dtype=nmslib.DistType.INT, data_type=nmslib.DataType.OBJECT_AS_STRING) 
index.addDataPointBatch(bert_vocab_toks)

30522

In [44]:
# Create an index
start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 25, 'indexThreadQty': 8, 'efConstruction': 200}
Indexing time = 4.700720


In [45]:
# Setting query-time parameters
efS = 1000
K=10
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 1000}


In [50]:
query_arr = ['fuuck', 'fcuk']
K=10

In [51]:
# Querying

query_qty = len(query_arr)
start = time.time() 
nbrs = index.knnQueryBatch(query_arr, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty))

kNN time total=0.006067 (sec), per query=0.003033 (sec), per query adjusted for thread number=0.024266 (sec)


In [52]:
nbrs[0][0], nbrs[0][1]

(array([ 6616, 10131,  4744,  5881, 18029,  6735,  9457, 11891, 22900,
         4248], dtype=int32),
 array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32))

In [53]:
for query_id in range(query_qty):
    print('Query:', query_id)
    qty = len(nbrs[query_id][0])
    for i in range(qty):
        print("word:", bert_vocab_toks[nbrs[query_id][0][i]], "distance: ", nbrs[query_id][1][i])
    print('===============')

Query: 0
word: fuck distance:  1
word: buck distance:  2
word: truck distance:  2
word: stuck distance:  2
word: tuck distance:  2
word: luck distance:  2
word: duck distance:  2
word: suck distance:  2
word: puck distance:  2
word: quick distance:  2
Query: 1
word: four distance:  2
word: cum distance:  2
word: cut distance:  2
word: cup distance:  2
word: cue distance:  2
word: fu distance:  2
word: faux distance:  2
word: cub distance:  2
word: flux distance:  2
word: feud distance:  2
